# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [ ]:
from ipywidgets import interact, interact_manual
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics import summary 
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
import plotly.graph_objects as go


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge, Def Var

In [ ]:
equity_bmk = 'SPTR'
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
returns = dm.get_equity_hedge_returns(equity_bmk, strat_drop_list = strat_drop_list)
weekly_ret=returns['Weekly'].copy()

##  Get Notional Weights 

In [ ]:
#notional_weights = dm.get_notional_weights(returns['Weekly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = util.get_df_weights(notional_weights, list(returns['Weekly'].columns))
fmt.get_notional_styler(df_weights)

## Get Raw Scores and Normalized Hedge Metrics

In [ ]:
hm_dict = summary.get_norm_hedge_metrics(weekly_ret, notional_weights, weighted=True)


## Display Data
To display data:
* Select which statistics you want to view **(stats)** -  Hedge Metrics or Normalized Data

In [ ]:
@interact
def display_hedge_metrics_or_normal( stats = ['Hedge Metrics','Normalized Data']):
    if stats == 'Hedge Metrics':
        return fmt.format_hm_to_normalize(hm_dict[stats])    
    else:
        return fmt.format_normalized_data(hm_dict[stats])


## Display Ranked Strategies vs Normalized Data

In [ ]:
df_norm = hm_dict['Normalized Data']
df_normal = df_norm.transpose()

#assign each strategy a symbol
symbols = plots.get_symbols(df_normal, unique=True)
color = plots.get_colors(df_normal, grey=False)

index = list(df_normal.index)
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 20, color = color[col]),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better",
                  plot_bgcolor='White')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.show()

# Ranked Stratagies with Defensive Universe

## Import Defensive Universe Returns Data

In [ ]:
def_uni = pd.read_excel(dm.EQUITY_HEDGE_DATA+'def_uni.xlsx',sheet_name='Weekly', index_col=0)
df_weighted_hedges = util.get_weighted_hedges(weekly_ret, notional_weights)
def_data = dm.merge_data_frames(df_weighted_hedges, def_uni)

## Get Defensive Universe Raw Scores and Normalized Hedge Metrics 

In [ ]:
#def_dict = summary.get_norm_hedge_metrics(def_data)
def_dict = {}
for key in hm_dict:
    def_dict[key] = pd.read_excel(dm.EQUITY_HEDGE_DATA+'def_uni_hedge_metrics.xlsx', sheet_name=key, index_col=0)

## Display Hedge Metrics and Normalized Data

In [ ]:
@interact
def display_hedge_metrics_or_normal( stats = ['Hedge Metrics','Normalized Data']):
    if stats == 'Hedge Metrics':
        return fmt.format_hm_to_normalize(def_dict[stats])    
    else:
        return fmt.format_normalized_data(def_dict[stats])

## Merge Defensive Unverse Normalized Data and Strategies Normalized Data

In [ ]:
merged_df=dm.merge_data_frames(def_normal,df_normal)
merged_df

## Display Ranked Strategies vs Normalized Data

In [ ]:
def_normal = def_data['Normalized Data']
def_normal = def_normal.transpose()

def_symbols = plots.get_symbols(def_normal, unique=False)
new_symbols= dm.merge_data_frames( def_symbols, symbols)
def_color = plots.get_colors(def_normal, grey=True)
new_color= dm.merge_data_frames( def_color, color)

#assign each strategy a symbol
index = list(merged_df.index)
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in merged_df.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=merged_df[col],
        marker=dict(size = 20, color = new_color[col]),
        marker_symbol = new_symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better",
                  plot_bgcolor='White')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.show()